<a href="https://colab.research.google.com/github/Titoffifee/CP_Dagestan/blob/main/transport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import recall_score, make_scorer, f1_score

df = pd.read_csv('drive/MyDrive/transport/train.csv')

In [3]:
y_labels = ['Погибло', 'Погибло детей', 'Ранено', 'Ранено детей']
delete = ['id', 'Километр', 'Метр', 'Дом', 'Дата', 'Время']

In [4]:
train, y0 = df.drop(delete + y_labels, axis=1), df[y_labels]

In [5]:
train['Вид ДТП'] = train['Вид ДТП'].fillna('Неизвестно')
train['Улица'] = train['Улица'].fillna('На шоссе')
train['Дорога'] = train['Дорога'].fillna('В городе')

In [6]:
test = pd.read_csv('drive/MyDrive/transport/test.csv')
test.info()
test = test.drop(delete, axis=1)
test['Вид ДТП'] = test['Вид ДТП'].fillna('Неизвестно')
test['Улица'] = test['Улица'].fillna('На шоссе')
test['Дорога'] = test['Дорога'].fillna('В городе')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6788 entries, 0 to 6787
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        6788 non-null   int64  
 1   Дата      6788 non-null   object 
 2   Время     6788 non-null   object 
 3   Место     6788 non-null   object 
 4   Улица     3852 non-null   object 
 5   Дом       3881 non-null   object 
 6   Дорога    1905 non-null   object 
 7   Километр  1904 non-null   float64
 8   Метр      1904 non-null   float64
 9   Вид ДТП   4988 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 530.4+ KB


In [ ]:
train.info()

In [60]:
obj = ['Место', 'Вид ДТП', 'Улица', 'Дорога']
max_value = 10
model = CatBoostClassifier(silent=True, cat_features=obj, random_seed=0,
                              loss_function='Logloss', auto_class_weights='Balanced')
index_name = dict()
index_name_list = []
for name in y_labels:
  index_name[name] = [name + str(i) for i in range(1, max_value+1)]
  index_name_list += index_name[name]

In [ ]:
y0

In [ ]:
def CreateY(ser):
  ans = pd.Series(dtype=int)
  for name in y_labels:
    r = pd.Series([0] * max_value, index=index_name, dtype=int)
    for i in range(1, ser[name] + 1):
      r[name + str(i)] = 1
    ans = pd.concat([ans, r])
  return ans
y = y0.apply(CreateY, axis=1)
y

In [56]:
p = pd.read_csv('drive/MyDrive/transport/solve.csv')

In [57]:
new_columns = p[y_labels].apply(CreateY, axis=1)

In [58]:
p = pd.concat([p['id'], new_columns], axis=1)
p

,id,Погибло1,Погибло2,Погибло3,Погибло4,Погибло5,Погибло6,Погибло7,Погибло8,Погибло9,...,Ранено детей1,Ранено детей2,Ранено детей3,Ранено детей4,Ранено детей5,Ранено детей6,Ранено детей7,Ранено детей8,Ранено детей9,Ранено детей10
0,490078911,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,490055448,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,490054440,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,490037847,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,490037049,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6783,490014641,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6784,490033115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6785,490008221,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6786,490119585,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
for name in y_labels:
  for i in range(1, max_value + 1):
    ind, last_ind = name + str(i), name + str(i-1)
    if i == 1:
      data_train = train
      data_test = test
    else:
      data_train = pd.concat([train, y[last_ind]], axis=1)
      data_test = pd.concat([test, p[last_ind]], axis=1)
    print(ind, 'WORK')
    if len(y[ind].unique()) == 1:
      break
    model.fit(data_train, y[ind])
    p[ind] = model.predict(data_test)
  print(name, 'OK')
p

Погибло1 WORK
Погибло2 WORK
Погибло3 WORK
Погибло4 WORK
Погибло OK
Погибло детей1 WORK
Погибло детей2 WORK
Погибло детей OK
Ранено1 WORK
Ранено2 WORK
Ранено3 WORK
Ранено4 WORK
Ранено5 WORK
Ранено6 WORK
Ранено7 WORK
Ранено8 WORK
Ранено9 WORK
Ранено OK
Ранено детей1 WORK
Ранено детей2 WORK
Ранено детей3 WORK
Ранено детей4 WORK
Ранено детей5 WORK
Ранено детей OK


,id,Погибло1,Погибло2,Погибло3,Погибло4,Погибло5,Погибло6,Погибло7,Погибло8,Погибло9,...,Ранено детей1,Ранено детей2,Ранено детей3,Ранено детей4,Ранено детей5,Ранено детей6,Ранено детей7,Ранено детей8,Ранено детей9,Ранено детей10
0,490078911,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,490055448,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,490054440,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,490037847,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,490037049,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6783,490014641,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6784,490033115,1,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
6785,490008221,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6786,490119585,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [63]:
def CreateRes(ser):
  ans = pd.Series([0] * 4, index=y_labels)
  for name in y_labels:
    r = 0
    for i in range(1, max_value + 1):
      if ser[name + str(i)] == 1:
        r = i
    ans[name] = r
  return ans

col = p[index_name_list].apply(CreateRes, axis=1)

In [66]:
p = pd.concat([p['id'], col], axis=1)

In [69]:
p.to_csv('drive/MyDrive/transport/ans.csv', index=False)